# CRISPR Screen Analysis

This script performs the following:
- preprocessing on fastq files
- MAGeCK-VISPR init to create config file
- edit config file
- execute MAGeCK-VISPR pipeline


Jupyter defaults to python, so each code block written in bash begins with:
%%bash

Jupyter has difficulty with conda, so each code block running MAGeCK-VISPR also begins with:

source /bobross/miniconda3/etc/profile.d/conda.sh

conda activate mageck-vispr

cd working_directory (you have to edit this in every code block)

# What is going on?
- This script assumes you already have your fastq files and a library of guide RNAs. More about lib in the config step.
- You will run an initialization command that will generate a config.yaml file (a file that tells MAGeCK your experimental parameters) and a Snakefile (a roadmap of conda programs to run for analysis). 
- You will edit the config file to reflect your control condition and library of guide RNAs.
- You will test the snakefile for missing input files and then execute the analysis.


# Create prelimConfig.txt and samples.txt
This code creates 2 preliminary config files that are used to inform the rest of the jupyter script. 

Instructions:
Edit your day0label, library, and list of sample names. 

- Library is a csv file of your grnas with their sequences and names formatted in a specific way. MAGeCK has common ones available for download on their SourceForge page (such as Brunello) or you can create your own by following the formatting. I will include the Brunello csv in the repository.
- day0label is the fastq prefix for your negative control or presort population. If your fastq file is control.fastq, your day0label will be control. 
- samples should be listed as the prefix to fastq. So for example, if you had fastqs control.fastq, replicate1.fastq and replicate2.fastq, your samples list would be 'control', 'replicate1', 'replicate2'

In [54]:
%cd /bobross/jvix/CRISPR/brunello
day0label = 'control'
library = 'Brunello.csv'
samples = ['control']

config = open("prelimConfig.txt", "w")
configelements = [day0label, library]
for line in configelements:
    config.write("%s\n" % line)
config.close()

samps = open("samples.txt", "w")
for line in samples:
    samps.write("%s\n" % line)
samps.close()
print('Done')

/bobross/jvix/CRISPR/brunello
Done


# Download Fastqs
Use this code if you don't have your fastq's yet. Edit the download command to suit your needs.

In [6]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
cd /bobross/jvix/CRISPR/brunello

FILES=$(cat samples.txt)
mkdir fastq
cd fastq


for i in ${FILES}
    do
        #download
        echo ${i}
    done
ls

mkdir: cannot create directory ‘fastq’: File exists


control
replicate1
replicate2


# Preprocessing
THIS CODE BLOCK IS MOST LIKELY NOT NECESSARY AND WILL CAUSE PROBLEMS IF YOU RUN IT. CHECK YOUR DATA FIRST.
This code block creates reverse-complement fastq files of your pre-existing fastq files. 

Instructions:

- Replace the file list with your fastq prefixes. For example, if you have a fastq titled C1X_R2.fastq, then you will add C1X_R2 to the list.
- Edit the code to reflect your input and output file locations.

In [7]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
cd /bobross/jvix/CRISPR/brunello

FILES=$(cat samples.txt)

for i in ${FILES}
    do
        #./bin/fastx_reverse_complement -Q 33 -i scratch/FASTQ/${i}.fastq -o scratch/rcfastq/${i}.fastq
        echo ${i}
    done
ls scratch/rcfastq

control
replicate1
replicate2


# MAGeCK-VISPR init
This code block generates a config.yaml file and displays its contents. It also generates a Snakefile.

Instructions:
- Replace the directory with your working directory
- Replace the list of reads with your fastq files

In [55]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
conda activate mageck-vispr
cd /bobross/jvix/CRISPR/brunello

# mageck-vispr init /path/to/working/directory --reads file/path/fastq1 file/path/fastq2 ...
mageck-vispr init /bobross/jvix/CRISPR/brunello --reads contorl.fastq  


#view config file
cat config.yaml

# General configuration:

# Path to library design file (csv or tab-separated txt format, columns: id, sequence, gene)
library: lib/library.csv
# Species to use for linkouts in VISPR (e.g. mus_musculus, homo_sapiens, ...)
species: homo_sapiens
# Genome assembly to use for linkouts in VISPR (e.g. hg19, hg38, mm9, mm10, ...)
assembly: hg38

# Configuration of knockout target display in VISPR
targets:
    # if screening genes, set this to true for proper linkouts to GeneMANIA and Ensembl in VISPR
    genes: true
    # file with genes to hide per default in VISPR (optional, one gene per line)
    #controls: ribosomal_genes.txt

# Configuration of sgRNAs
sgrnas:
    # estimate sgRNA knockout efficiency during EM-procedure of MAGeCK-MLE
    update-efficiency: false
    # trim the 5 prime end to get rid of barcode sequences in the reads
    # if a number (instead of AUTO) is specified, use quotes; for example:
    # trim-5: "0"
    trim-5: AUTO
    # specify the length of the sgRNAs (without 

# Edit config file
This code block uses python and preliminConfig.txt to edit your config file. 

In [57]:

import os
import pandas as pd

config = pd.read_table('config.yaml', header=None, delimiter=None)[0].to_list()
day0lib = pd.read_table('prelimConfig.txt', header=None, delimiter=None)[0].to_list()
samples = pd.read_table('samples.txt', header=None, delimiter=None)[0].to_list()

#print(day0lib)
#print(samples)
config[2] = day0lib[1]
config[58+2*(len(samples))] = 'day0label:'+ day0lib[0]
print(len(samples))
os.remove('config.yaml')

fp = open('config.yaml', 'w')
for line in config:
    print(line)
    fp.write("%s\n" % line)
fp.close()
print('Done')

1
# General configuration:
# Path to library design file (csv or tab-separated txt format, columns: id, sequence, gene)
Brunello.csv
# Species to use for linkouts in VISPR (e.g. mus_musculus, homo_sapiens, ...)
species: homo_sapiens
# Genome assembly to use for linkouts in VISPR (e.g. hg19, hg38, mm9, mm10, ...)
assembly: hg38
# Configuration of knockout target display in VISPR
targets:
    # if screening genes, set this to true for proper linkouts to GeneMANIA and Ensembl in VISPR
    genes: true
    # file with genes to hide per default in VISPR (optional, one gene per line)
    #controls: ribosomal_genes.txt
# Configuration of sgRNAs
sgrnas:
    # estimate sgRNA knockout efficiency during EM-procedure of MAGeCK-MLE
    update-efficiency: false
    # trim the 5 prime end to get rid of barcode sequences in the reads
    # if a number (instead of AUTO) is specified, use quotes; for example:
    # trim-5: "0"
    trim-5: AUTO
    # specify the length of the sgRNAs (without PAM sequence)

# Perform a dry run followed by analysis
Instructions:
- Change the directory to your working directory
- Make sure the first code block doesn't get angry with you about nonexistent files or other errors.
- Run the second code block. This could take up to a few hours. It usually hangs out at ~62% depending on your sample number for a million years. Don't be concerned.

In [4]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
conda activate mageck-vispr



snakemake -n


Building DAG of jobs...
Job stats:
job             count    min threads    max threads
------------  -------  -------------  -------------
all                 1              1              1
fastqc              3              1              1
mageck_count        1              1              1
mageck_mle          1              1              1
mageck_rra          1              1              1
vispr               1              1              1
total               8              1              1


[Fri Jul  7 09:08:51 2023]
rule fastqc:
    input: rcfastq/S1H_R2.fastq
    output: results/qc/S1H_R2_0
    log: logs/fastqc/S1H_R2_0.log
    jobid: 6
    reason: Updated input files: rcfastq/S1H_R2.fastq
    wildcards: replicate=S1H_R2_0
    resources: tmpdir=/tmp


[Fri Jul  7 09:08:51 2023]
rule fastqc:
    input: rcfastq/C1X_R2.fastq
    output: results/qc/C1X_R2_0
    log: logs/fastqc/C1X_R2_0.log
    jobid: 5
    reason: Updated input files: rcfastq/C1X_R2.fastq
    wildcards: replica

In [5]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
conda activate mageck-vispr
snakemake --cores 8

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 8
Rules claiming more threads will be scaled down.
Job stats:
job             count    min threads    max threads
------------  -------  -------------  -------------
all                 1              1              1
fastqc              3              1              1
mageck_count        1              1              1
mageck_mle          1              1              1
mageck_rra          1              1              1
vispr               1              1              1
total               8              1              1

Select jobs to execute...

[Fri Jul  7 09:09:02 2023]
rule fastqc:
    input: rcfastq/C1X_R2.fastq
    output: results/qc/C1X_R2_0
    log: logs/fastqc/C1X_R2_0.log
    jobid: 5
    reason: Updated input files: rcfastq/C1X_R2.fastq
    wildcards: replicate=C1X_R2_0
    resources: tmpdir=/tmp


[Fri Jul  7 09:09:02 2023]
rule fastqc:
    input: rcfastq/S1H_R2.fastq
    output: results/qc/S1H_R2_0
    log

Analysis complete for S2L_R2.fastq


[Fri Jul  7 09:09:54 2023]
Finished job 7.
1 of 8 steps (12%) done


Analysis complete for C1X_R2.fastq


[Fri Jul  7 09:25:45 2023]
Finished job 5.
2 of 8 steps (25%) done


Analysis complete for S1H_R2.fastq


[Fri Jul  7 09:26:04 2023]
Finished job 6.
3 of 8 steps (38%) done
[Fri Jul  7 09:28:58 2023]
Finished job 3.
4 of 8 steps (50%) done
Select jobs to execute...

[Fri Jul  7 09:28:59 2023]
rule mageck_mle:
    input: results/count/all.count.txt, /dev/null
    output: results/test/mle.gene_summary.txt, results/test/mle.sgrna_summary.txt
    log: logs/mageck/test/mle.log
    jobid: 2
    reason: Missing output files: results/test/mle.gene_summary.txt, results/test/mle.sgrna_summary.txt; Input files updated by another job: results/count/all.count.txt
    wildcards: experiment=mle
    resources: tmpdir=/tmp


[Fri Jul  7 09:28:59 2023]
rule mageck_rra:
    input: results/count/all.count.txt
    output: results/test/mle.rra.gene_summary.txt, results/test/mle.rra.sgrna_summary.txt, results/test/mle.rra.S1H_R2_vs_C1X_R2.sgrna_summary.txt, results/test/mle.rra.S2L_R2_vs_C1X_R2.sgrna_summary.txt
    log: logs/mageck/test/mle.rra.log
    jobid: 4
    reason: Missing output files: results/test/mle

#  Run VISPR
You MUST manually stop the code block when you are done!!! Closing jupyter without stopping the code block will leave the process running in the background on the server.

In [1]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
conda activate mageck-vispr
nohup vispr server results/mle.vispr.yaml --host 132.198.80.178 --port 8079

Starting server.

Open:  go to 132.198.80.178:8079 in your browser.
Note: Safari and Internet Explorer are currently unsupported.
Close: hit Ctrl-C in this terminal.


Loading data.
 * Serving Flask app 'vispr.server'
 * Debug mode: off
Error while terminating subprocess (pid=3358220): 
